In [1]:
import pandas as pd
import numpy as np

league_2017_ml = pd.read_csv('csv/nhl.20162017/neural_refine.20162017.csv')
league_2017_ml.head()

,Team,Name,Playoffs?,Conference,Division,Season End,Season Type,Game,Points,Corsi Close,...,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,Goal Differential Close,Fenwick Close Differential
0,ANA,Ducks,Playoff,Western,Pacific,2016,Regular,82,103,53.68,...,385,349,48.63,195,206,33.97,71,138,4,322
1,ARI,Coyotes,DNQ,Western,Pacific,2016,Regular,82,78,45.65,...,295,355,43.02,154,204,40.68,48,70,-20,-366
2,BOS,Bruins,DNQ,Eastern,Atlantic,2016,Regular,82,93,49.87,...,392,372,41.82,161,224,37.89,72,118,1,0
3,BUF,Sabres,DNQ,Eastern,Atlantic,2016,Regular,82,81,47.02,...,340,376,41.22,115,164,36.89,45,77,-21,-245
4,CAR,Hurricanes,DNQ,Eastern,Metropolitan,2016,Regular,82,86,51.55,...,424,357,47.80,163,178,38.46,45,72,-28,81


### preprocessing 20162017 NHL

In [2]:
data = league_2017_ml.values
Team = data[:,0]
Name = data[:,1]
Playoffs = data[:,2]
Conference = data[:,3]
Division = data[:,4]
SeasonType = data[:,6]

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [4]:
label_encoder.fit(Team)
encoded_team = label_encoder.transform(Team)

In [5]:
label_encoder.fit(Name)
encoded_name = label_encoder.transform(Name)

In [6]:
label_encoder.fit(Playoffs)
encoded_playoffs = label_encoder.transform(Playoffs)

In [7]:
label_encoder.fit(Conference)
encoded_conference = label_encoder.transform(Conference)

In [8]:
label_encoder.fit(Division)
encoded_division = label_encoder.transform(Division)

In [9]:
label_encoder.fit(SeasonType)
encoded_season = label_encoder.transform(SeasonType)

In [10]:
concatenate = {'Team': encoded_team,
               'Name': encoded_name, 
               'Playoffs': encoded_playoffs,
               'Conference': encoded_conference,
               'Division': encoded_division,
               'SeasonType': encoded_season,
               }

encoding_df = pd.DataFrame(data = concatenate)

In [11]:
encoding_df.head()

,Conference,Division,Name,Playoffs,SeasonType,Team
0,1,3,10,1,0,0
1,1,3,8,0,0,1
2,0,0,4,0,0,2
3,0,0,25,0,0,3
4,0,2,13,0,0,4


In [ ]:
# encoding_df.to_csv('csv/nhl.20162017/encoded_refine.20162017.csv', index = False)

### Neural Network, training with 20162017 NHL

In [12]:
encoded_ml = pd.read_csv('csv/nhl.20162017/encoded_refine.20162017.csv')
encoded_ml.head()

,Conference,Division,Name,Playoffs,SeasonType,Team,Season End,Game,Points,Corsi Close,...,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,Goal Differential Close,Fenwick Close Differential
0,1,3,10,1,0,0,2016,82,103,53.68,...,385,349,48.63,195,206,33.97,71,138,4,322
1,1,3,8,0,0,1,2016,82,78,45.65,...,295,355,43.02,154,204,40.68,48,70,-20,-366
2,0,0,4,0,0,2,2016,82,93,49.87,...,392,372,41.82,161,224,37.89,72,118,1,0
3,0,0,25,0,0,3,2016,82,81,47.02,...,340,376,41.22,115,164,36.89,45,77,-21,-245
4,0,2,13,0,0,4,2016,82,86,51.55,...,424,357,47.80,163,178,38.46,45,72,-28,81


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

Using TensorFlow backend.


In [14]:
X = encoded_ml.drop('Playoffs', axis = 1)
y = encoded_ml['Playoffs']

In [15]:
X.shape

(240, 73)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 1, stratify = y)

In [17]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [19]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [20]:
from keras.models import Sequential
from keras.layers import Dense

#Create model and add layers
model = Sequential()
model.add(Dense(units = 100, activation = 'relu', input_dim = 73))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))

In [21]:
#compile and fit the model
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs = 5,
    shuffle = True,
    verbose = 2)

Epoch 1/5
 - 1s - loss: 0.6444 - acc: 0.6278
Epoch 2/5
 - 0s - loss: 0.3910 - acc: 0.8500
Epoch 3/5
 - 0s - loss: 0.2779 - acc: 0.9222
Epoch 4/5
 - 0s - loss: 0.2164 - acc: 0.9333
Epoch 5/5
 - 0s - loss: 0.1685 - acc: 0.9667


In [22]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled,
    y_test_categorical,
    verbose = 2)

print(f'Normal Neural Network| Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network| Loss: 0.2488068570693334, Accuracy: 0.8166666626930237


In [23]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f'Predicted classes: {prediction_labels}')
print(f'Actual Labels: {list(y_test[:5])}')

Predicted classes: [0 1 1 0 1]
Actual Labels: [0, 1, 1, 1, 0]


### saving a trained model with .h5

In [24]:
#saving the trained model using the HDF5 binary format with the extension .h5

model.save('saved_model/neural_refine_nhl.h5')

### evaluating the loaded model

In [25]:
#load the model

from keras.models import load_model
predict_team = load_model('saved_model/neural_refine_nhl.h5')

In [26]:
model_loss, model_accuracy = predict_team.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.1459907967183325, Accuracy: 0.9666666666666667


In [27]:
model_loss, model_accuracy = predict_team.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.2488068570693334, Accuracy: 0.8166666626930237


### preprocessing 2018 NHL

In [28]:
league_2018_ml = pd.read_csv('csv/nhl.2018/neural_refine.2018.csv')
league_2018_ml.head()

,Team,Name,Conference,Division,Season End,Season Type,Cumulative Games,Points,Corsi Close,Fenwick Close,...,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,Goal Differential Close,Fenwick Close Differential
0,ANA,Ducks,Western,Pacific,2018,Regular,60,69,47.88,48.30,...,253,263,41.94,130,180,40.00,78,117,25,-23
1,ARI,Coyotes,Western,Pacific,2018,Regular,60,44,47.44,45.97,...,349,300,45.57,149,178,45.36,44,53,-9,-213
2,BOS,Bruins,Eastern,Atlantic,2018,Regular,60,82,52.40,52.10,...,280,262,47.83,121,132,44.19,38,48,38,172
3,BUF,Sabres,Eastern,Atlantic,2018,Regular,60,45,47.17,46.89,...,185,243,37.97,71,116,35.53,27,49,-47,-137
4,CAR,Hurricanes,Eastern,Metropolitan,2018,Regular,60,64,54.69,54.27,...,318,229,51.97,145,134,53.06,52,46,-8,245


In [29]:
data  = league_2018_ml.values
Team = data[:,0]
Name = data[:,1]
Conference = data[:,2]
Division = data[:,3 ]
SeasonType = data[:,5]

In [30]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [31]:
label_encoder.fit(Team)
encoded_team = label_encoder.transform(Team)

In [32]:
label_encoder.fit(Name)
encoded_name = label_encoder.transform(Name)

In [33]:
label_encoder.fit(Conference)
encoded_conference = label_encoder.transform(Conference)

In [34]:
label_encoder.fit(Division)
encoded_division = label_encoder.transform(Division)

In [35]:
label_encoder.fit(SeasonType)
encoded_season = label_encoder.transform(SeasonType)

In [36]:
concatenate = {'Team': encoded_team,
               'Name': encoded_name,
               'Conference': encoded_conference,
               'Division': encoded_division,
               'SeasonType': encoded_season,
               }

encoding_df = pd.DataFrame(data = concatenate)

In [37]:
encoding_df.head()

,Conference,Division,Name,SeasonType,Team
0,1,3,10,0,0
1,1,3,8,0,1
2,0,0,4,0,2
3,0,0,26,0,3
4,0,2,14,0,4


In [ ]:
# encoding_df.to_csv('csv/nhl.2018/encoded_refine.2018.csv', index = False)

In [38]:
#read the training data into a dataframe
nhl_2018 = pd.read_csv('csv/nhl.2018/encoded_refine.2018.csv')
reverse_nhl_2018 = pd.read_csv('csv/nhl.2018/neural_refine.2018.csv')

In [39]:
reverse_nhl_2018.head()

,Team,Name,Conference,Division,Season End,Season Type,Cumulative Games,Points,Corsi Close,Fenwick Close,...,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,Goal Differential Close,Fenwick Close Differential
0,ANA,Ducks,Western,Pacific,2018,Regular,60,69,47.88,48.30,...,253,263,41.94,130,180,40.00,78,117,25,-23
1,ARI,Coyotes,Western,Pacific,2018,Regular,60,44,47.44,45.97,...,349,300,45.57,149,178,45.36,44,53,-9,-213
2,BOS,Bruins,Eastern,Atlantic,2018,Regular,60,82,52.40,52.10,...,280,262,47.83,121,132,44.19,38,48,38,172
3,BUF,Sabres,Eastern,Atlantic,2018,Regular,60,45,47.17,46.89,...,185,243,37.97,71,116,35.53,27,49,-47,-137
4,CAR,Hurricanes,Eastern,Metropolitan,2018,Regular,60,64,54.69,54.27,...,318,229,51.97,145,134,53.06,52,46,-8,245


In [40]:
nhl_2018.head()

,Conference,Division,Name,SeasonType,Team,Season End,Cumulative Games,Points,Corsi Close,Fenwick Close,...,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,Goal Differential Close,Fenwick Close Differential
0,1,3,10,0,0,2018,60,69,47.88,48.30,...,253,263,41.94,130,180,40.00,78,117,25,-23
1,1,3,8,0,1,2018,60,44,47.44,45.97,...,349,300,45.57,149,178,45.36,44,53,-9,-213
2,0,0,4,0,2,2018,60,82,52.40,52.10,...,280,262,47.83,121,132,44.19,38,48,38,172
3,0,0,26,0,3,2018,60,45,47.17,46.89,...,185,243,37.97,71,116,35.53,27,49,-47,-137
4,0,2,14,0,4,2018,60,64,54.69,54.27,...,318,229,51.97,145,134,53.06,52,46,-8,245


In [41]:
#convert dataframe to a numpy array for Keras

season_2018 = nhl_2018.values
season_2018

array([[   1.,    3.,   10., ...,  117.,   25.,  -23.],
       [   1.,    3.,    8., ...,   53.,   -9., -213.],
       [   0.,    0.,    4., ...,   48.,   38.,  172.],
       ..., 
       [   1.,    3.,   13., ...,   76.,   16.,   87.],
       [   1.,    1.,   16., ...,   95.,   34.,  133.],
       [   0.,    2.,    7., ...,  119.,   28., -148.]])

In [42]:
season_2018.shape

(31, 73)

### Neural Network, testing with 20162017 NHL

In [43]:
#loading the model

from keras.models import load_model
model = load_model('saved_model/neural_refine_nhl.h5')

In [44]:
season_2018.shape

(31, 73)

### 1 = Playoffs | 0 = DNQ


In [45]:
neural_2018 = model.predict_classes(season_2018)

In [46]:
concatenate = {'Playoffs': neural_2018,
               'Name': reverse_nhl_2018['Name']}

In [47]:
results_2018 = pd.DataFrame(data = concatenate)
results_2018

,Name,Playoffs
0,Ducks,1
1,Coyotes,0
2,Bruins,1
3,Sabres,0
4,Hurricanes,0
5,Blue Jackets,0
6,Flames,0
7,Blackhawks,1
8,Avalanche,1
9,Stars,1


In [48]:
results_2018.to_csv('nhl_refined.2018.csv', index = False)